## ENRICH Algorithm

Iterative method to retrieve the proper order of elements with lowest maximum similarity from the remainder to an initial subgroup. Using trivial example.

In [1]:
import csv
import pandas as pd
import numpy as np
import numba as nb
import math
import json

import matplotlib.pyplot as plt

## Example

- convert diagonal from similarity 1 to 0 
- A and B are in our initial subgroup
- select element from the remainder, based on similarity values with A, B

In [2]:
all_names = np.asarray(["A", "B", "C", "D", "E", "F", "G", "H"])
my_list = [None]*6
my_matrix = np.asarray([[1, 0.16, 0.15, 0.14, 0.10, 0.004, 0.07, 0.50], 
                        [0.16, 1, 0.13, 0.12, 0.08, 0.06, 0.133, 0.20], 
                        [0.15, 0.13, 1, 0.11, 0.09, 0.21, 0.22, 0.23], 
                        [0.14, 0.12, 0.11, 1, 0.71, 0.06, 0.18, 0.19], 
                        [0.10, 0.08, 0.09, 0.71, 1, 0.24, 0.25, 0.06],
                        [0.004, 0.06, 0.21, 0.06, 0.24, 1, 0.44, 0.42], 
                        [0.07, 0.133, 0.22, 0.18, 0.25, 0.44, 1, 0.001], 
                        [0.50, 0.20, 0.23, 0.19, 0.06, 0.42, 0.001, 1]])

np.fill_diagonal(my_matrix, 0)
my_matrix

array([[0.   , 0.16 , 0.15 , 0.14 , 0.1  , 0.004, 0.07 , 0.5  ],
       [0.16 , 0.   , 0.13 , 0.12 , 0.08 , 0.06 , 0.133, 0.2  ],
       [0.15 , 0.13 , 0.   , 0.11 , 0.09 , 0.21 , 0.22 , 0.23 ],
       [0.14 , 0.12 , 0.11 , 0.   , 0.71 , 0.06 , 0.18 , 0.19 ],
       [0.1  , 0.08 , 0.09 , 0.71 , 0.   , 0.24 , 0.25 , 0.06 ],
       [0.004, 0.06 , 0.21 , 0.06 , 0.24 , 0.   , 0.44 , 0.42 ],
       [0.07 , 0.133, 0.22 , 0.18 , 0.25 , 0.44 , 0.   , 0.001],
       [0.5  , 0.2  , 0.23 , 0.19 , 0.06 , 0.42 , 0.001, 0.   ]])

In [3]:
initial_names = ["A", "B"]

### matrix version

order should be: F, D, C, G, H, E

In [4]:
namesDF = pd.DataFrame({"sim_name": all_names})

In [5]:
namesDF["sim_name"].isin(initial_names).values

array([ True,  True, False, False, False, False, False, False])

In [66]:
%%timeit
names = ["A", "B"]
for j in range(6):
    mask = namesDF["sim_name"].isin(names).values    
    new_pick_index = np.argmin(np.max(my_matrix[np.ix_(mask, ~mask)], axis=0))
    names += [np.asarray(namesDF["sim_name"][~mask])[new_pick_index]]

2.34 ms ± 69.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
names = ["A", "B"]
for j in range(6):
    mask = namesDF["sim_name"].isin(names).values
    
    # remainder similarity with subgroup
    print(my_matrix[np.ix_(mask, ~mask)])
    
    # max of remainder similarity with subgroup
    print(np.max(my_matrix[np.ix_(mask, ~mask)], axis=0))
    
    # min of the max of remainder similarity with subgroup
    new_pick_index = np.argmin(np.max(my_matrix[np.ix_(mask, ~mask)], axis=0))
    names += [np.asarray(namesDF["sim_name"][~mask])[new_pick_index]]
    
    print(names)

[[0.15  0.14  0.1   0.004 0.07  0.5  ]
 [0.13  0.12  0.08  0.06  0.133 0.2  ]]
[0.15  0.14  0.1   0.06  0.133 0.5  ]
['A', 'B', 'F']
[[0.15  0.14  0.1   0.07  0.5  ]
 [0.13  0.12  0.08  0.133 0.2  ]
 [0.21  0.06  0.24  0.44  0.42 ]]
[0.21 0.14 0.24 0.44 0.5 ]
['A', 'B', 'F', 'D']
[[0.15  0.1   0.07  0.5  ]
 [0.13  0.08  0.133 0.2  ]
 [0.11  0.71  0.18  0.19 ]
 [0.21  0.24  0.44  0.42 ]]
[0.21 0.71 0.44 0.5 ]
['A', 'B', 'F', 'D', 'C']
[[0.1   0.07  0.5  ]
 [0.08  0.133 0.2  ]
 [0.09  0.22  0.23 ]
 [0.71  0.18  0.19 ]
 [0.24  0.44  0.42 ]]
[0.71 0.44 0.5 ]
['A', 'B', 'F', 'D', 'C', 'G']
[[0.1   0.5  ]
 [0.08  0.2  ]
 [0.09  0.23 ]
 [0.71  0.19 ]
 [0.24  0.42 ]
 [0.25  0.001]]
[0.71 0.5 ]
['A', 'B', 'F', 'D', 'C', 'G', 'H']
[[0.1 ]
 [0.08]
 [0.09]
 [0.71]
 [0.24]
 [0.25]
 [0.06]]
[0.71]
['A', 'B', 'F', 'D', 'C', 'G', 'H', 'E']


### updated matrix version

order should be: F, D, C, G, H, E

In [13]:
namesDF = pd.DataFrame({"sim_name": all_names})
names = np.asarray(namesDF["sim_name"])
names

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], dtype=object)

In [14]:
initial_names = np.zeros((8), dtype=object)
initial_names[0] = "A"
initial_names[1] = "B"
initial_names

array(['A', 'B', 0, 0, 0, 0, 0, 0], dtype=object)

In [15]:
%%timeit
initial_names = np.zeros((8), dtype=object)
initial_names[0] = "A"
initial_names[1] = "B"

for j in range(6):
    mask = np.isin(initial_names, names)    
    new_pick_index = np.argmin(np.max(my_matrix[np.ix_(mask, ~mask)], axis=0))
    new_name = names[~mask][new_pick_index]

    new_name_index = np.where(names == new_name)
    initial_names[new_name_index] = new_name

376 µs ± 15.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [16]:
initial_names = np.zeros((8), dtype=object)
initial_names[0] = "A"
initial_names[1] = "B"

for j in range(6):
    mask = np.isin(initial_names, names)
    print(my_matrix[np.ix_(mask, ~mask)])
    print(np.max(my_matrix[np.ix_(mask, ~mask)], axis=0))
    
    new_pick_index = np.argmin(np.max(my_matrix[np.ix_(mask, ~mask)], axis=0))
    new_name = names[~mask][new_pick_index]

    new_name_index = np.where(names == new_name)
    initial_names[new_name_index] = new_name
    print(initial_names)

[[0.15  0.14  0.1   0.004 0.07  0.5  ]
 [0.13  0.12  0.08  0.06  0.133 0.2  ]]
[0.15  0.14  0.1   0.06  0.133 0.5  ]
['A' 'B' 0 0 0 'F' 0 0]
[[0.15  0.14  0.1   0.07  0.5  ]
 [0.13  0.12  0.08  0.133 0.2  ]
 [0.21  0.06  0.24  0.44  0.42 ]]
[0.21 0.14 0.24 0.44 0.5 ]
['A' 'B' 0 'D' 0 'F' 0 0]
[[0.15  0.1   0.07  0.5  ]
 [0.13  0.08  0.133 0.2  ]
 [0.11  0.71  0.18  0.19 ]
 [0.21  0.24  0.44  0.42 ]]
[0.21 0.71 0.44 0.5 ]
['A' 'B' 'C' 'D' 0 'F' 0 0]
[[0.1   0.07  0.5  ]
 [0.08  0.133 0.2  ]
 [0.09  0.22  0.23 ]
 [0.71  0.18  0.19 ]
 [0.24  0.44  0.42 ]]
[0.71 0.44 0.5 ]
['A' 'B' 'C' 'D' 0 'F' 'G' 0]
[[0.1   0.5  ]
 [0.08  0.2  ]
 [0.09  0.23 ]
 [0.71  0.19 ]
 [0.24  0.42 ]
 [0.25  0.001]]
[0.71 0.5 ]
['A' 'B' 'C' 'D' 0 'F' 'G' 'H']
[[0.1 ]
 [0.08]
 [0.09]
 [0.71]
 [0.24]
 [0.25]
 [0.06]]
[0.71]
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H']


### w/ numba

order should be: F, D, C, G, H, E

future improvements:
- update np.max w/ numba 
    - currently no optional arguments are allowed

In [17]:
namesDF = pd.DataFrame({"sim_name": all_names})
names = np.asarray(namesDF["sim_name"])
names

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], dtype=object)

In [39]:
@nb.jit(nopython=True, cache=False)
def numba_initial_bool(size):
    return np.zeros((size), dtype=np.bool_)

@nb.jit(nopython=True, cache=False)
def numba_nonzero(mask):
    return mask.nonzero()[0]

@nb.jit(nopython=True, cache=False)
def numba_ix(arr, rows, cols):
    """
    taken directly from https://github.com/numba/numba/issues/5894
    numba compatible implementation of arr[np.ix_(rows, cols)] for 2D arrays.
    """
    one_d_index = np.zeros(len(rows) * len(cols), dtype=np.int32)
    for i, r in enumerate(rows):
        start = i * len(cols)
        one_d_index[start: start + len(cols)] = cols + arr.shape[1] * r

    arr_1d = arr.reshape((arr.shape[0] * arr.shape[1], 1))
    slice_1d = np.take(arr_1d, one_d_index)
    return slice_1d.reshape((len(rows), len(cols)))

In [41]:
%%timeit
mask = numba_initial_bool(8)
mask[0] = True
mask[1] = True

for j in range(6):
    remainder_index = np.argmin(np.max(numba_ix(my_matrix, numba_nonzero(mask), numba_nonzero(~mask)), axis=0))    
    remainders = numba_nonzero(~mask)
    mask[remainders[remainder_index]] = True

68.8 µs ± 4.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [43]:
mask = numba_initial_bool(8)
mask[0] = True
mask[1] = True

for j in range(6):
    print(my_matrix[np.ix_(mask, ~mask)])
    print(np.max(my_matrix[np.ix_(mask, ~mask)], axis=0))

    remainder_index = np.argmin(np.max(numba_ix(my_matrix, numba_nonzero(mask), numba_nonzero(~mask)), axis=0))  
    remainders = numba_nonzero(~mask)
    mask[remainders[remainder_index]] = True
    print(names[mask])

[[0.15  0.14  0.1   0.004 0.07  0.5  ]
 [0.13  0.12  0.08  0.06  0.133 0.2  ]]
[0.15  0.14  0.1   0.06  0.133 0.5  ]
['A' 'B' 'F']
[[0.15  0.14  0.1   0.07  0.5  ]
 [0.13  0.12  0.08  0.133 0.2  ]
 [0.21  0.06  0.24  0.44  0.42 ]]
[0.21 0.14 0.24 0.44 0.5 ]
['A' 'B' 'D' 'F']
[[0.15  0.1   0.07  0.5  ]
 [0.13  0.08  0.133 0.2  ]
 [0.11  0.71  0.18  0.19 ]
 [0.21  0.24  0.44  0.42 ]]
[0.21 0.71 0.44 0.5 ]
['A' 'B' 'C' 'D' 'F']
[[0.1   0.07  0.5  ]
 [0.08  0.133 0.2  ]
 [0.09  0.22  0.23 ]
 [0.71  0.18  0.19 ]
 [0.24  0.44  0.42 ]]
[0.71 0.44 0.5 ]
['A' 'B' 'C' 'D' 'F' 'G']
[[0.1   0.5  ]
 [0.08  0.2  ]
 [0.09  0.23 ]
 [0.71  0.19 ]
 [0.24  0.42 ]
 [0.25  0.001]]
[0.71 0.5 ]
['A' 'B' 'C' 'D' 'F' 'G' 'H']
[[0.1 ]
 [0.08]
 [0.09]
 [0.71]
 [0.24]
 [0.25]
 [0.06]]
[0.71]
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H']
